In [2]:
'''
Exploratory data analysis: 
This .jpynb removed null values, admin error data, and compiled latitude/longitude from street addresses
using geopy library

The final csv output of this file is "cleaned_crime.csv"
The required input is "Crime.csv"
The datset source can be found here: https://www.kaggle.com/datasets/melissamonfared/cambridge-crime-data-2009-2024?resource=download
'''
# Import libraries
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Read Crime.csv from Cambridge MA gov websitr
crime_df = pd.read_csv('Crime.csv')

# print all column names of the DataFrame
print(crime_df.columns.tolist())
print(crime_df['Location'])
#['File Number', 'Date of Report', 'Crime Date Time', 'Crime', 'Reporting Area', 'Neighborhood', 'Location']


['File Number', 'Date of Report', 'Crime Date Time', 'Crime', 'Reporting Area', 'Neighborhood', 'Location']
0                           100 OTIS ST, Cambridge, MA
1                        400 RINDGE AVE, Cambridge, MA
2                        400 RINDGE AVE, Cambridge, MA
3                        0 NORUMBEGA ST, Cambridge, MA
4                    FIFTH ST & GORE ST, Cambridge, MA
                             ...                      
95918               100 BISHOP ALLEN DR, Cambridge, MA
95919    MASSACHUSETTS AVE & PEABODY ST, Cambridge, MA
95920                     0 COLUMBIA ST, Cambridge, MA
95921                       0 FOSTER PL, Cambridge, MA
95922             200 Alewife Brook Pky, Cambridge, MA
Name: Location, Length: 95923, dtype: object


In [3]:
# Check for unique crime types - these will be the possible variables
unique_crime_types = crime_df['Crime'].unique()

print("Unique crime types:")
print(unique_crime_types)
unique_crime_types.shape

Unique crime types:
['Threats' 'Auto Theft' 'Hit and Run' 'Larceny (Misc)' 'OUI'
 'Aggravated Assault' 'Commercial Break' 'Street Robbery' 'Housebreak'
 'Shoplifting' 'Forgery' 'Simple Assault' 'Warrant Arrest' 'Disorderly'
 'Larceny from Building' 'Mal. Dest. Property' 'Larceny from MV'
 'Trespassing' 'Larceny from Person' 'Missing Person'
 'Larceny from Residence' 'Harassment' 'Liquor Possession/Sale'
 'Flim Flam' 'Phone Calls' 'Larceny of Bicycle' 'Annoying & Accosting'
 'Drugs' 'Indecent Exposure' 'Larceny of Plate' 'Sex Offender Violation'
 'Counterfeiting' 'Rec. Stol. Property' 'Commercial Robbery' 'Kidnapping'
 'Drinking in Public' 'Larceny of Services' 'Peeping & Spying' 'Homicide'
 'Extortion/Blackmail' 'Stalking' 'Weapon Violations' 'Prostitution'
 'Arson' 'Embezzlement' 'Admin Error' 'Violation of R.O.'
 'Violation of H.O.' 'Domestic Dispute' 'Gambling' 'Accident'
 'Noise Complaint' 'Suspicious Package' 'Taxi Violation']


(54,)

# Cleaning null values and "admin error"s

In [4]:
# Sum the number of rows with at least 1 null value
null_values = crime_df.isnull().any()

rows_with_nulls = crime_df.isnull().any(axis=1).sum()
print(f"Number of rows with at least one null value: {rows_with_nulls}")
# 311 is insignificant: remove from dataframe 
print(f"the size of total datapoints is {crime_df.shape}")


Number of rows with at least one null value: 311
the size of total datapoints is (95923, 7)


In [5]:
crime_df = crime_df.dropna()
rows_with_nulls = crime_df.isnull().any(axis=1).sum()
print(f"Number of rows with at least one null value is now: {rows_with_nulls}")

Number of rows with at least one null value is now: 0


In [6]:
# Remove all Crimes that are "Admin Error" and check crime_df again
crime_df = crime_df[crime_df['Crime'] != 'Admin Error']


# Generating latitude and longitude from location

In [7]:
'''This block uses geopy to generate lat/long pairs from the street addresses supplied by crime.csv'''
run = 0 #if function to stop geopy requests from going off
if run == 1:
    # Initialize geolocater
    geolocator = Nominatim(user_agent="eitan")

    # Set rate limit
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)

    # Function to get latitude and longitude
    def get_lat_long(address):
        '''
        get_lat_long takes a street address and outputs latitude and longitude as a tuple
        input: a street address as a string
        output: two floats, lat and longitude
        '''
        # Added printout to confirm function is still running
        print(f"Processing address: {address}")
        location = geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)

    # Remove any duplicate addresses to speed up lat/long process and then merge in post
    unique_locations = crime_df[['Location']].drop_duplicates()

    # Apply the geocoding function to the 'Location' column
    unique_locations['Coordinates'] = unique_locations['Location'].apply(get_lat_long)
    unique_locations[['Latitude', 'Longitude']] = pd.DataFrame(unique_locations['Coordinates'].tolist(), index=unique_locations.index)


    unique_locations.to_csv('Crime_with_coordinates.csv', index=False)

In [8]:
coords_df = pd.read_csv('Crime_with_coordinates.csv')
coords_df.shape
# There are ~5000 unique locations from the crime.csv data set. 

(5144, 4)

In [14]:
# Map lat and longitude to each address for crime_df using merge

crime_with_coords_df = pd.read_csv('Crime_with_coordinates.csv')
merged_df = pd.merge(crime_df, crime_with_coords_df[['Location', 'Latitude', 'Longitude']], on='Location', how='left')

merged_df.to_csv('cleaned_crime.csv', index=False)
merged_df
# cleaned_crime.csv is now the output csv to work with

,File Number,Date of Report,Crime Date Time,Crime,Reporting Area,Neighborhood,Location,Latitude,Longitude
0,2009-01323,02/21/2009 09:53:00 AM,02/21/2009 09:20 - 09:30,Threats,105.0,East Cambridge,"100 OTIS ST, Cambridge, MA",42.370427,-71.082645
1,2009-01324,02/21/2009 09:59:00 AM,02/20/2009 22:30 - 02/21/2009 10:00,Auto Theft,1109.0,North Cambridge,"400 RINDGE AVE, Cambridge, MA",42.393764,-71.139481
2,2009-01327,02/21/2009 12:32:00 PM,02/19/2009 21:00 - 02/21/2009 12:00,Hit and Run,1109.0,North Cambridge,"400 RINDGE AVE, Cambridge, MA",42.393764,-71.139481
3,2009-01331,02/21/2009 03:05:00 PM,02/21/2009 15:00 - 15:10,Larceny (Misc),1303.0,Strawberry Hill,"0 NORUMBEGA ST, Cambridge, MA",42.375145,-71.153088
4,2009-01346,02/22/2009 05:02:00 AM,02/22/2009 05:02,OUI,105.0,East Cambridge,"FIFTH ST & GORE ST, Cambridge, MA",NaN,NaN
...,...,...,...,...,...,...,...,...,...
92750,2024-03755,05/07/2024 01:13:00 PM,05/04/2024 12:00 - 18:00,Larceny from MV,411.0,Area 4,"100 BISHOP ALLEN DR, Cambridge, MA",42.365860,-71.102518
92751,2024-03756,05/07/2024 02:41:00 PM,05/07/2024 14:40 - 14:41,Accident,611.0,Mid-Cambridge,"MASSACHUSETTS AVE & PEABODY ST, Cambridge, MA",42.374898,-71.118521
92752,2024-03777,05/07/2024 08:13:00 PM,05/07/2024 15:00 - 19:15,Larceny of Bicycle,411.0,Area 4,"0 COLUMBIA ST, Cambridge, MA",42.370400,-71.095941
92753,2024-03806,05/08/2024 04:09:00 PM,05/07/2024 04:00 - 04:05,Larceny from MV,1005.0,West Cambridge,"0 FOSTER PL, Cambridge, MA",42.375982,-71.129503


In [19]:
# Check if there's any Locations not in Cambridge (check if all values in the 'Location' column end with "'Cambridge, MA'")
all_end_with_MA = crime_df['Location'].str.endswith('Cambridge, MA').all()

# Print the result
if all_end_with_MA:
    print("All addresses end with 'Cambridge, MA'.")
else:
    print("Not all addresses end with 'Cambridge, MA'.")

# Optionally, print the rows that do not end with 'MA'
addresses_not_ending_with_MA = crime_df[~crime_df['Location'].str.endswith('MA')]
print("Addresses not ending with 'MA':")
print(addresses_not_ending_with_MA)

All addresses end with 'Cambridge, MA'.
Addresses not ending with 'MA':
Empty DataFrame
Columns: [File Number, Date of Report, Crime Date Time, Crime, Reporting Area, Neighborhood, Location]
Index: []


In [20]:
# Sum the number of rows with at least 1 null value
null_values = merged_df.isnull().any()

rows_with_nulls = merged_df.isnull().any(axis=1).sum()
print(f"Number of rows with at least one null value: {rows_with_nulls}")

unique_2 = merged_df[['Location']].drop_duplicates()
print(unique_2.size)

Number of rows with at least one null value: 12345
5079
